<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/nilm_rnn_fre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [3]:
uploaded = files.upload()
'''Load data'''
WHE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_WHE.csv']))['P']
#WHE_data = pd.read_csv('Electricity_WHE.csv')['P']

Saving Electricity_WHE.csv to Electricity_WHE.csv


In [5]:
uploaded = files.upload()
FRE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_FRE.csv']))['P']
#CWE_data = pd.read_csv('Electricity_CWE.csv')['P']

Saving Electricity_FRE.csv to Electricity_FRE.csv


In [0]:
window_size=60

dataX_raw = create_dataset(WHE_data.as_matrix(), window_size)

#S0:0-210,S1:210-400

fre_Y_raw = FRE_data[window_size-1:].values.reshape([FRE_data.shape[0]-window_size+1,1])
dataX = np.concatenate([dataX_raw[0:472500,:],dataX_raw[475500:,:]],axis=0)
fre_Y = np.concatenate([fre_Y_raw[0:472500,:],fre_Y_raw[475500:,:]],axis=0)
categorized_fre_Y = np.ones(fre_Y.shape)*2
categorized_fre_Y[[np.where((fre_Y>=0)&(fre_Y<=210))[0]],:]=0
categorized_fre_Y[[np.where((fre_Y>210)&(fre_Y<=400))[0]],:]=1

encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
freY_1hot = encoder.fit_transform(categorized_fre_Y)

X_train, X_test, y_train, y_test = train_test_split(dataX, freY_1hot, test_size=0.1, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train.astype(float))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
max_indice = np.argmax(freY_1hot,1)
df = pd.Series(max_indice)
df.value_counts()

0    1039538
1       8590
2         13
dtype: int64

In [0]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:7500,1:7500,2:0})
X_train, y_train = tl.fit_resample(X_train, y_train)
whole_train = np.concatenate([X_train,y_train],axis=1)
#from imblearn.under_sampling import EditedNearestNeighbours
#enn = EditedNearestNeighbours()
#X_train, y_train = enn.fit_resample(X_train, y_train)

#from imblearn.under_sampling import NeighbourhoodCleaningRule
#ncr = NeighbourhoodCleaningRule()
#X_train, y_train = ncr.fit_resample(X_train, y_train)

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=128
unit_num=64

In [11]:
'''RNN Model Definition'''
tf.reset_default_graph()
''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, window_size,1],name='x')
tf_y = tf.placeholder(tf.int32, [None, 3],name='y')

lstm_cell =tf.contrib.rnn.BasicLSTMCell(num_units=unit_num,name='lstm_cell')
outputs, (h_c, h_n) = tf.nn.dynamic_rnn(
    lstm_cell,                   # cell you have chosen
    tf_x,                      # input
    initial_state=None,         # the initial hidden state
    dtype=tf.float32,           # must given if set initial_state = None
    time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)
l1 = tf.layers.dense(outputs[:, -1, :],64,activation=tf.nn.leaky_relu,name='l1')
l2 = tf.layers.dense(l1,32,activation=tf.nn.leaky_relu,name='l2')
l3 = tf.layers.dense(l2,16,activation=tf.nn.leaky_relu,name='l3')
l4 = tf.layers.dense(l3,8,activation=tf.nn.leaky_relu,name='l4')
pred = tf.layers.dense(l4,3,activation=tf.nn.relu,name='pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.


In [0]:
sess = tf.Session()
#sess.run(init_op)
saver.restore(sess, 'my_net_fre/save_fre_rnn.ckpt')

flag = False
for j in range(0,300):
    print('###iteration: '+str(j)+'###')
    batch_index = np.random.choice(15000,batch_size)
    batch_train = whole_train[batch_index,:]
    batch_X = batch_train[:,:-3].reshape([batch_size,window_size,1])
    batch_y = batch_train[:,-3:]
    sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
    cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
    acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
    acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y:y_test})
    print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
    print('Test Acc=%.2f'% acc_test)
    
    pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
    y_lables_argmax = tf.argmax(tf_y,axis=1)  
    y_pred_argmax = tf.argmax(pre,axis=1)
    confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
    #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
    print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
    if acc_train>=0.99:
      print(j)
      flag = True
      break
    if(flag==True):
        print(flag)
        break
    if(j//100==0):
      save_path = saver.save(sess, "my_net_fre/save_fre_rnn.ckpt")
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from my_net_fre/save_fre_rnn.ckpt
###iteration: 0###
train loss= 0.3264, Acc=0.87
Test Acc=0.91
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
[[51 10  0]
 [ 7 60  0]
 [ 0  0  0]]
###iteration: 1###
train loss= 0.4009, Acc=0.86
Test Acc=0.93
[[58  6  0]
 [12 52  0]
 [ 0  0  0]]
###iteration: 2###
train loss= 0.2770, Acc=0.92
Test Acc=0.92
[[59  5  0]
 [ 5 59  0]
 [ 0  0  0]]
###iteration: 3###
train loss= 0.2707, Acc=0.90
Test Acc=0.89
[[59  6  0]
 [ 7 56  0]
 [ 0  0  0]]
###iteration: 4###
train loss= 0.2683, Acc=0.92
Test Acc=0.89
[[49  8  0]
 [ 2 69  0]
 [ 0  0  0]]
###iteration: 5###
train loss= 0.3051, Acc=0.90
Test Acc=0.89
[[52  6  0]
 [ 7 63  0]
 [ 0  0  0]]
###iteration: 6###
train loss= 0.3377, Acc=0.87
Test Acc=0.90
[[54  9  0]
 [ 8 57  0]
 [ 0  0  0]]
###iteration: 7###
train loss= 0.3842, Acc=0.88
Test 

In [0]:
#!mkdir my_net_fre
mv checkpoint save_fre_rnn.ckpt.data-00000-of-00001 save_fre_rnn.ckpt.index save_fre_rnn.ckpt.meta my_net_fre